In [3]:
import pandas as pd,seaborn as sns,matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import json
import requests
from bs4 import BeautifulSoup
from modules.functions import get_schedule,create_home_and_away_simple_dataframe,plot_game_trends
from modules.functions import calculate_possessions,get_agg_boxscore,get_game_timestamp_half
from IPython.display import clear_output
import time

In [4]:
get_schedule("Purdue")

,DATE,OPPONENT,GAME_ID
2,"Tue, Nov 8",Milwaukee,401483405
3,"Fri, Nov 11",Austin Peay,401483406
4,"Tue, Nov 15",Marquette,401479696
5,"Thu, Nov 24",West Virginia,401483331
6,"Fri, Nov 25",Gonzaga,401486748
7,"Sun, Nov 27",Duke,401486751
8,"Wed, Nov 30",@ Florida State,401479679
10,"Sun, Dec 4",Minnesota,401483407
11,"Wed, Dec 7",Hofstra,401484843
12,"Sat, Dec 10",@ Nebraska,401483408


In [11]:
game_id = 401484836

n_tries = 0

try:
    test_df
except NameError as e:
    test_df = pd.DataFrame()

last_reported_time = None

while n_tries < 10:

    
    timestamp,half = get_game_timestamp_half(game_id)

    if timestamp != last_reported_time:
        last_reported_time = timestamp
        n_tries = 0
        clear_output()
        last_reported_time = timestamp
        temp_df = get_agg_boxscore(game_id = game_id,disp = True)
        temp_df = temp_df.transpose().rename_axis("Team").reset_index()
        temp_df['Half'] = half
        temp_df['Timestamp'] = timestamp
        test_df = pd.concat([test_df,temp_df])
        create_home_and_away_simple_dataframe(game_id,disp = True)
        time.sleep(15)
    else:
        time.sleep(25)
        n_tries +=1

,Purdue,Minnesota
PTS,89,70
FGM,33,28
FGA,65,59
3PM,7,9
3PA,17,26
FTM,16,5
FTA,18,8
OREB,10,3
DREB,28,16
TO,7,8


Player  PTS  FGM  FGA  3PM  3PA  FTM  FTA  OREB  DREB  \
Minnesota 0  J. Ola-Joseph   10    5   10    0    0    0    0     1     2   
          1      D. Garcia    6    3    8    0    3    0    0     0     1   
          2      J. Battle   21    8   17    5   10    0    0     0     1   
          3      J. Henley    5    2    5    0    3    1    2     0     1   
          4      T. Cooper   15    6    9    3    4    0    0     0     5   
          5       P. Payne    2    1    1    0    0    0    1     0     1   
          6  B. Carrington    9    2    7    1    5    4    5     1     5   
          7     W. Ramberg    0    0    0    0    0    0    0     1     0   
          8     T. Samuels    2    1    2    0    1    0    0     0     0   
          9           Team   70   28   59    9   26    5    8     3    16   

             REB  AST  STL  BLK  TO  PF  PTS/FGA Position  
Minnesota 0    3    0    1    0   0   4     1.00        F  
          1    1    1    0    0   0   5     0.75        F  
          2    1    2    0    0   2   1     1.24        F  
          3    1    0    0    0   1   0     1.00        F  
          4    5    8    0    1   2   2     1.67        G  
          5    1    2    0    4   2   2     2.00        F  
          6    6    0    1    0   0   3     1.29        G  
          7    1    0    0    0   0   0     0.00        G  
          8    0    1    1    0   1   1     1.00        G  
          9   19   14    3    5   8  18     9.95

Player  PTS  FGM  FGA  3PM  3PA  FTM  FTA  OREB  DREB  REB  \
Purdue 0         C. Furst   10    4    5    2    2    0    0     1     1    2   
       1          Z. Edey   31   11   23    0    0    9   10     8    14   22   
       2         B. Smith    0    0    4    0    2    0    0     0     2    2   
       3         F. Loyer   20    7   11    2    4    4    4     0     1    1   
       4        E. Morton    3    1    2    1    2    0    0     0     1    1   
       5  T. Kaufman-Renn   10    5   10    0    2    0    1     1     5    6   
       6       B. Waddell    0    0    0    0    0    0    0     0     0    0   
       7        B. Newman   13    4    8    2    5    3    3     0     1    1   
       8   D. Jenkins Jr.    2    1    2    0    0    0    0     0     3    3   
       9             Team   89   33   65    7   17   16   18    10    28   38   

          AST  STL  BLK  TO  PF  PTS/FGA Position  
Purdue 0    1    1    0   2   3     2.00        F  
       1    0    0    1   2   2     1.35        C  
       2    4    1    0   0   2     0.00        G  
       3    8    1    0   0   0     1.82        G  
       4    0    0    1   0   1     1.50        G  
       5    3    1    0   1   3     1.00        F  
       6    1    0    0   0   0     0.00        F  
       7    2    0    0   1   0     1.62        G  
       8    3    1    0   1   2     1.00        G  
       9   22    5    2   7  13    10.29

In [ ]:
plot_game_trends(test_df,half = half)